In [17]:
import pandas as pd

def get_poblacion()->pd.DataFrame:
    """
    Obtiene de los archivos
        aior\test\data\poblacion_por_municipios\DCD-area-proypoblacion-Mun-1985-1994 (1).xlsx
        aior\test\data\poblacion_por_municipios\DCD-area-proypoblacion-Mun-1995-2004 (2).xlsx
        aior\test\data\poblacion_por_municipios\DCD-area-proypoblacion-Mun-2005-2019 (2).xlsx
        aior\test\data\poblacion_por_municipios\DCD-area-proypoblacion-Mun-2020-2035-ActPostCOVID-19 (2).xlsx
    los datos de población por municipio y año.

    Returns
    -------
    DataFrame
        DataFrame con los datos de población por municipio y año.
    """
    
    # Carga de datos
    df_1985_1994 = pd.read_excel('poblacion_por_municipios\\DCD-area-proypoblacion-Mun-1985-1994 (1).xlsx', dtype='object')
    df_1995_2004 = pd.read_excel('poblacion_por_municipios\\DCD-area-proypoblacion-Mun-1995-2004 (2).xlsx', dtype='object')
    df_2005_2019 = pd.read_excel('poblacion_por_municipios\\DCD-area-proypoblacion-Mun-2005-2019 (2).xlsx', dtype='object')
    # TODO: #7 Anotar que en este archivo 2005-2019 hay un error en las columnas, se debe cambiar dpmp y mpio de lugar
    df_2020_2035 = pd.read_excel('poblacion_por_municipios\\DCD-area-proypoblacion-Mun-2020-2035-ActPostCOVID-19 (2).xlsx', dtype='object')
    # Concatenación de los datos
    df = pd.concat([df_1985_1994, df_1995_2004, df_2005_2019, df_2020_2035], ignore_index=True)
    # turn column names into lowercase
    df.columns = df.columns.str.lower()
    columns = df.columns # ['dp', 'dpnom', 'dpmp', 'mpio', 'año', 'área geográfica', 'población']
    area_uniques = df['área geográfica'].unique() # ['Cabecera Municipal' 'Centros Poblados y Rural Disperso' 'Total']
    # Turn area uniques into columns for the same dp and dpmp and año
    df = df.pivot_table(index=['dp', 'dpnom', 'dpmp', 'año'], columns='área geográfica', values='población').reset_index()
    df.columns = df.columns.str.lower()
    # turn dpmp values into string
    df['dpmp'] = df['dpmp'].astype(str)
    df['dpmp'] = df['dpmp'].apply(lambda x: "0" + x if len(x) != 5 else x)
    return df

def pivot_per_municipio(df:pd.DataFrame)->pd.DataFrame:
    # TODO: #8 Hacer el docstring
    cabecera = df.pivot_table(index=['dp', 'dpnom', 'dpmp'], columns='año', values='cabecera municipal').reset_index()
    centros_poblados = df.pivot_table(index=['dp', 'dpnom', 'dpmp'], columns='año', values='centros poblados y rural disperso').reset_index()
    total = df.pivot_table(index=['dp', 'dpnom', 'dpmp'], columns='año', values='total').reset_index()
    return total, cabecera, centros_poblados
poblacion = get_poblacion()
por_municipio_total, por_municipio_cabecera, por_municipio_centros_poblados = pivot_per_municipio(poblacion)
por_municipio_total.to_csv('poblacion_por_municipio_total.csv',index=False)
por_municipio_cabecera.to_csv('poblacion_por_municipio_cabecera.csv', index=False)
por_municipio_centros_poblados.to_csv('poblacion_por_municipio_centros_poblados.csv', index=False)

In [18]:
# columns = año	dp	dpnom	dpmp	mpio	1985	1986	1987	1988	1989	1990	...	2026	2027	2028	2029	2030	2031	2032	2033	2034	2035
# show the rows with NaN values
# read municipios.csv and get all the data as objects, not numbers
municipios = pd.read_csv('municipios.csv', dtype='object')
# Change the column 	c_digo_dane_del_municipio to dpmp
municipios = municipios.rename(columns={'c_digo_dane_del_municipio': 'dpmp'})
# Show which dpmp values are not in the population data
not_in_population = municipios[~municipios['dpmp'].isin(poblacion['dpmp'])]
display(not_in_population)


,region,c_digo_dane_del_departamento,departamento,dpmp,municipio,search_query,place_id,licence,osm_type,osm_id,lat,lon,category,type,place_rank,importance,addresstype,name,display_name,boundingbox
64,Región Eje Cafetero - Antioquia,5,Antioquia,0540,La Unión,"La Unión, Antioquia, Colombia",270295596,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,1307235,5.9737744,-75.3613093,boundary,administrative,12,0.44986236905806365,town,La Unión,"La Unión, Oriente, Antioquia, RAP del Agua y l...","['5.8576303', '6.0453384', '-75.4275839', '-75..."
103,Región Llano,50,Meta,05037,Uribe,"Uribe, Meta, Colombia",267212825,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,7229667,3.2413296,-74.3510657,boundary,administrative,12,0.3454990317530525,town,Uribe,"Uribe, Ariari, Meta, RAP (Especial) Central, C...","['2.5074080', '3.7875189', '-74.9339283', '-73..."
142,Región Caribe,13,Bolívar,01314,Calamar,"Calamar, Bolívar, Colombia",267993723,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,1341497,10.2526122,-74.9146691,boundary,administrative,12,0.377343921991154,town,Calamar,"Calamar, Dique, Bolívar, RAP Caribe, Colombia","['10.1015032', '10.3657713', '-75.0926681', '-..."
143,Región Caribe,13,Bolívar,01316,Cantagallo,"Cantagallo, Bolívar, Colombia",267495205,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,1335188,7.3780007,-73.9150265,boundary,administrative,12,0.3672320103116836,town,Cantagallo,"Cantagallo, Magdalena Medio, Bolívar, RAP Cari...","['7.0012603', '7.4311084', '-74.2940587', '-73..."
148,Región Caribe,13,Bolívar,01343,Magangué,"Magangué, Bolívar, Colombia",266067728,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,1326231,9.2412097,-74.7567413,boundary,administrative,12,0.40898930833454095,city,Magangué,"Magangué, Mojana, Bolívar, RAP Caribe, Colombia","['8.8192442', '9.4656588', '-74.9476687', '-74..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,Región Centro Sur,73,Tolima,07377,Suárez,"Suárez, Tolima, Colombia",267350795,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,4047422,4.0484889,-74.833034,boundary,administrative,12,0.3605079208970536,town,Suarez,"Suarez, Suroriente, Tolima, RAP (Especial) Cen...","['3.8927587', '4.2661587', '-74.8781776', '-74..."
1111,Región Llano,50,Meta,05015,Castilla la Nueva,"Castilla la Nueva, Meta, Colombia",268169570,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,11375320,3.8275343,-73.6892805,boundary,administrative,12,0.3158021943488055,town,Castilla La Nueva,"Castilla La Nueva, Piedemonte, Meta, RAP (Espe...","['3.7060937', '3.9210089', '-73.7400450', '-73..."
1113,Región Pacífico,27,Chocó,02781,Unión Panamericana,"Unión Panamericana, Chocó, Colombia",268551173,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,1350459,5.2813079,-76.6293417,boundary,administrative,12,0.3481598362912266,village,Las Ánimas,"Las Ánimas, San Juan, Chocó, RAP Pacífico, Col...","['5.1903448', '5.4051634', '-76.7264709', '-76..."
1114,Región Caribe,47,Magdalena,04757,Pueblo Viejo,"Pueblo Viejo, Magdalena, Colombia",268443034,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,5744725551,10.7876917,-73.9183907,place,hamlet,20,0.25000999999999995,hamlet,Pueblo Viejo de Sentay,"Pueblo Viejo de Sentay, Ciénaga, Magdalena, RA...","['10.7676917', '10.8076917', '-73.9383907', '-..."
